In [1]:
# pip install nba_api
from nba_api.stats.static import players
import pandas as pd

pd.set_option('display.max_columns', None)

In [2]:
#Get advanced stat data from basketball reference 
#pip install basketball_reference_web_scraper
from basketball_reference_web_scraper import client

In [3]:
#If player name = "player", populate shot chart detail parameter "player_id" with 'id' from:
# player = input("Enter an NBA players full name i.e 'Michael Jordan' or 'Stephen Curry':\nYou picked ")

from operator import itemgetter 

def get_players():
    player_dic = players.get_players()
    return player_dic

def get_player_name_and_id():
    '''https://www.geeksforgeeks.org/python-get-values-of-particular-key-in-list-of-dictionaries/'''
    player_dic = get_players()
#     name_and_id = [ x['id'] for x in player_dic ] 
    name_and_id = list(map(itemgetter('id','full_name'), player_dic))
    name_and_id = pd.DataFrame(name_and_id)
    name_and_id.rename(columns={0:'id',1:'name'}, inplace=True)
    name_and_id['id'] = name_and_id['id'].astype(str)
    
    return name_and_id

get_player_name_and_id()

,id,name
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz
2,76003,Kareem Abdul-Jabbar
3,51,Mahmoud Abdul-Rauf
4,1505,Tariq Abdul-Wahad
...,...,...
4496,1627790,Ante Zizic
4497,78647,Jim Zoet
4498,78648,Bill Zopf
4499,1627826,Ivica Zubac


In [4]:
def players_advanced_season(year):
    '''This function scrapes all players advanced stats from basketball reference.
       It also combines the basketball reference advanced stats with the nba pypi api, by merging the two on and id'''
    br_data = client.players_advanced_season_totals(season_end_year=year)
    
    br_data = pd.DataFrame(br_data)

    br_data["true_shooting_percentage"] = br_data["true_shooting_percentage"] * 100
    br_data['year'] = year
    
    stats_api_id = get_player_name_and_id()

    advanced_stat_df = pd.merge(br_data, stats_api_id, how = "left",on="name")

#     advanced_stat_df = advanced_stat_df[['year','id','name','slug','player_efficiency_rating','true_shooting_percentage','win_shares','box_plus_minus','value_over_replacement_player']]
    
    return advanced_stat_df

def year_plus_one(x=2020):
    '''This function returns all players advanced stats in the league from any years that you choose, from basketball reference'''
    for idx , year in enumerate(range(2019,x+1)):
        if idx == 0:
            advanced_stat_df = players_advanced_season(year)
        else:
            temp_data = players_advanced_season(year)
            advanced_stat_df = advanced_stat_df.append(temp_data)           
    return advanced_stat_df

In [5]:
advanced_stats = year_plus_one(x= 2019)

In [6]:
advanced_stats

,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,year,id
0,abrinal01,Álex Abrines,[Position.SHOOTING_GUARD],25,Team.OKLAHOMA_CITY_THUNDER,31,588,6.3,50.7,0.809,0.083,0.9,7.8,4.2,4.3,1.3,0.9,7.9,12.2,0.1,0.6,0.6,0.053,-3.7,0.4,-3.3,-0.2,False,2019,NaN
1,acyqu01,Quincy Acy,[Position.POWER_FORWARD],28,Team.PHOENIX_SUNS,10,123,2.9,37.9,0.833,0.556,2.7,20.1,11.3,8.2,0.4,2.7,15.2,9.2,-0.1,0.0,-0.1,-0.022,-7.6,-0.5,-8.1,-0.2,False,2019,203112
2,adamsja01,Jaylen Adams,[Position.POINT_GUARD],22,Team.ATLANTA_HAWKS,34,428,7.6,47.4,0.673,0.082,2.6,12.3,7.4,19.8,1.5,1.0,19.7,13.5,-0.1,0.2,0.1,0.011,-3.8,-0.5,-4.3,-0.2,False,2019,1629121
3,adamsst01,Steven Adams,[Position.CENTER],25,Team.OKLAHOMA_CITY_THUNDER,80,2669,18.5,59.1,0.002,0.361,14.7,14.8,14.7,6.6,2.0,2.4,12.6,16.4,5.1,4.0,9.1,0.163,0.7,0.4,1.1,2.1,False,2019,203500
4,adebaba01,Bam Adebayo,[Position.CENTER],21,Team.MIAMI_HEAT,82,1913,17.9,62.3,0.031,0.465,9.2,24.0,16.6,14.2,1.8,3.0,17.1,15.8,3.4,3.4,6.8,0.171,-0.4,2.2,1.8,1.8,False,2019,1628389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,zellety01,Tyler Zeller,[Position.CENTER],29,Team.ATLANTA_HAWKS,2,11,1.5,0.0,0.500,0.000,18.7,39.1,28.7,10.6,0.0,0.0,0.0,7.4,0.0,0.0,0.0,-0.013,-8.7,-1.6,-10.2,0.0,False,2019,203092
619,zellety01,Tyler Zeller,[Position.CENTER],29,Team.MEMPHIS_GRIZZLIES,4,82,19.4,64.0,0.000,0.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,0.2,0.1,0.3,0.191,-2.3,-1.9,-4.3,0.0,False,2019,203092
620,zizican01,Ante Žižić,[Position.CENTER],22,Team.CLEVELAND_CAVALIERS,59,1082,16.2,59.0,0.000,0.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,1.7,0.3,2.0,0.087,-1.1,-2.1,-3.2,-0.3,False,2019,NaN
621,zubaciv01,Ivica Zubac,[Position.CENTER],21,Team.LOS_ANGELES_LAKERS,33,516,19.5,63.3,0.000,0.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,1.1,0.6,1.8,0.165,-0.3,0.1,-0.2,0.2,False,2019,1627826


In [7]:
positions = advanced_stats['positions']

In [8]:
[print(position[0]) for position in positions]

Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.CENTER
Position.SMALL_FORWARD
Position.SHOOTING_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.CENTER
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.POINT_GUARD
Position.POINT_GUARD
Position.CENTER
Position.POINT_GUARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.POWER_FORWARD
P

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [11]:
# Take values from series
positions = advanced_stats['positions']
# Grab n object from the list object 
advanced_stats["pos"] = [print(position[0]) for position in positions]

# Use the split function on "." 
advanced_stats
# advanced_stats['positions'] = advanced_stats['positions'].map(lambda x:x.split('.')[1])
# advanced_stats['positions'] = advanced_stats['positions'].astype('str')

# Grab first element from split[1]

# Return list in corresponding column

Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.CENTER
Position.SMALL_FORWARD
Position.SHOOTING_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.CENTER
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.POWER_FORWARD
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.POINT_GUARD
Position.POINT_GUARD
Position.CENTER
Position.POINT_GUARD
Position.POWER_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.SMALL_FORWARD
Position.POINT_GUARD
Position.CENTER
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.SHOOTING_GUARD
Position.POWER_FORWARD
P

,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,assist_percentage,steal_percentage,block_percentage,turnover_percentage,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,year,id,pos
0,abrinal01,Álex Abrines,[Position.SHOOTING_GUARD],25,Team.OKLAHOMA_CITY_THUNDER,31,588,6.3,50.7,0.809,0.083,0.9,7.8,4.2,4.3,1.3,0.9,7.9,12.2,0.1,0.6,0.6,0.053,-3.7,0.4,-3.3,-0.2,False,2019,NaN,None
1,acyqu01,Quincy Acy,[Position.POWER_FORWARD],28,Team.PHOENIX_SUNS,10,123,2.9,37.9,0.833,0.556,2.7,20.1,11.3,8.2,0.4,2.7,15.2,9.2,-0.1,0.0,-0.1,-0.022,-7.6,-0.5,-8.1,-0.2,False,2019,203112,None
2,adamsja01,Jaylen Adams,[Position.POINT_GUARD],22,Team.ATLANTA_HAWKS,34,428,7.6,47.4,0.673,0.082,2.6,12.3,7.4,19.8,1.5,1.0,19.7,13.5,-0.1,0.2,0.1,0.011,-3.8,-0.5,-4.3,-0.2,False,2019,1629121,None
3,adamsst01,Steven Adams,[Position.CENTER],25,Team.OKLAHOMA_CITY_THUNDER,80,2669,18.5,59.1,0.002,0.361,14.7,14.8,14.7,6.6,2.0,2.4,12.6,16.4,5.1,4.0,9.1,0.163,0.7,0.4,1.1,2.1,False,2019,203500,None
4,adebaba01,Bam Adebayo,[Position.CENTER],21,Team.MIAMI_HEAT,82,1913,17.9,62.3,0.031,0.465,9.2,24.0,16.6,14.2,1.8,3.0,17.1,15.8,3.4,3.4,6.8,0.171,-0.4,2.2,1.8,1.8,False,2019,1628389,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,zellety01,Tyler Zeller,[Position.CENTER],29,Team.ATLANTA_HAWKS,2,11,1.5,0.0,0.500,0.000,18.7,39.1,28.7,10.6,0.0,0.0,0.0,7.4,0.0,0.0,0.0,-0.013,-8.7,-1.6,-10.2,0.0,False,2019,203092,None
619,zellety01,Tyler Zeller,[Position.CENTER],29,Team.MEMPHIS_GRIZZLIES,4,82,19.4,64.0,0.000,0.643,12.1,12.5,12.3,6.2,0.6,3.5,10.0,21.8,0.2,0.1,0.3,0.191,-2.3,-1.9,-4.3,0.0,False,2019,203092,None
620,zizican01,Ante Žižić,[Position.CENTER],22,Team.CLEVELAND_CAVALIERS,59,1082,16.2,59.0,0.000,0.399,10.6,22.8,16.4,7.7,0.6,1.8,13.6,18.2,1.7,0.3,2.0,0.087,-1.1,-2.1,-3.2,-0.3,False,2019,NaN,None
621,zubaciv01,Ivica Zubac,[Position.CENTER],21,Team.LOS_ANGELES_LAKERS,33,516,19.5,63.3,0.000,0.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,1.1,0.6,1.8,0.165,-0.3,0.1,-0.2,0.2,False,2019,1627826,None


In [ ]:
advanced_stats.positions.unique

In [ ]:
teams = advanced_stats['team']
teams

In [ ]:
# mapping = {'[Position.POINT_GUARD]': 1, '[Position.SHOOTING_GUARD]': 2, '[Position.SMALL_FORWARD]': 3, '[Position.POWER_FORWARD]': 4, '[Position.CENTER]':5}
# advanced_stats['positions'] = advanced_stats['positions'].map(mapping)
# advanced_stats

In [ ]:
# advanced_stats['positions'].mask(advanced_stats['positions'] == '[Position.SHOOTING_GUARD]', 'SG', inplace=False)
# advanced_stats

In [ ]:
# advanced_stats.loc[(advanced_stats.positions == '[Position.SHOOTING_GUARD]'),'positions']='Shooting_Guard'
# advanced_stats

In [ ]:
# del advanced_stats['team']
# del advanced_stats['positions']

In [ ]:
advanced_stats['team'] = advanced_stats['team'].astype('str')
# advanced_stats['team'] = advanced_stats['team'].str[5:]
advanced_stats['team'] = advanced_stats['team'].map(lambda x:x.split('.')[1])
# advanced_stats
advanced_stats

In [ ]:
advanced_stats.dtypes

In [ ]:
advanced_stats

# Put Advanced Stats into Database

In [ ]:
# pip install psycopg2

In [ ]:
from sqlalchemy import create_engine
from module import password

In [ ]:
connection_string = f'postgres:{password}@localhost:5432/nba_br_advanced_stats_db'

engine = create_engine(f'postgresql://{connection_string}')
# , pool_pre_ping=True

In [ ]:
engine

In [ ]:
#Confirm tables
engine.table_names()

In [ ]:
advanced_stats.to_sql(name='advanced_stats', con=engine, if_exists='replace', index=False)